In [1]:
import datasets
import pandas as pd
import os
os.environ['CUDA_VISIBLE_DEVICES']='0'

/data/conda_envs/project1/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Training

In [2]:
# git clone https://github.com/ZIZUN/korean-malicious-comments-dataset

df= pd.read_csv('./korean-malicious-comments-dataset/Dataset.csv')

In [3]:
df

,content\tlable
0,이종석 한효주 나오는 드라마 이후로 드라마 안봤다. 2년전인가?? 좀 신선했었지. ...
1,씨바알..노무노무 술프노... 오늘 저녁은 꽂등심이다ㅠㅜ\t0
2,짱깨 꺼라ㅡ패쓰\t0
3,그들의 사생활 ~ 고인이된 설리를 위해서라도 모두 조용하길 지금 누굴 탓한다고 무슨...
4,아무리 법이 뭣같아도 무슨 자격으로 개인의 신상정보를 불특정 다수에게 공개하는지 도...
...,...
9995,이거 보다 더한 막장의 드라마도 넋놓고 보면서 무슨?\t1
9996,머...제네시스..올라프에디션?\t1
9997,항상 여자로 ㅡ ㅡ 저지랄 들이야 남자들 벗겨봐라\t0
9998,첫번째 이유가 ㄹㅇ 맞지특히나 뉴스나 기사 몇줄읽은 학생들끼리 대화해도 서로 존나 ...


In [4]:
# sep='\t' 를 통해서 탭 구분자로 불러오기
df= pd.read_csv('./korean-malicious-comments-dataset/Dataset.csv',sep='\t')

In [5]:
df

,content,lable
0,이종석 한효주 나오는 드라마 이후로 드라마 안봤다. 2년전인가?? 좀 신선했었지. ...,0.0
1,씨바알..노무노무 술프노... 오늘 저녁은 꽂등심이다ㅠㅜ,0.0
2,짱깨 꺼라ㅡ패쓰,0.0
3,그들의 사생활 ~ 고인이된 설리를 위해서라도 모두 조용하길 지금 누굴 탓한다고 무슨...,1.0
4,아무리 법이 뭣같아도 무슨 자격으로 개인의 신상정보를 불특정 다수에게 공개하는지 도...,1.0
...,...,...
9995,이거 보다 더한 막장의 드라마도 넋놓고 보면서 무슨?,1.0
9996,머...제네시스..올라프에디션?,1.0
9997,항상 여자로 ㅡ ㅡ 저지랄 들이야 남자들 벗겨봐라,0.0
9998,첫번째 이유가 ㄹㅇ 맞지특히나 뉴스나 기사 몇줄읽은 학생들끼리 대화해도 서로 존나 ...,0.0


In [6]:
df.columns=['text','label']



In [7]:
df

,text,label
0,이종석 한효주 나오는 드라마 이후로 드라마 안봤다. 2년전인가?? 좀 신선했었지. ...,0.0
1,씨바알..노무노무 술프노... 오늘 저녁은 꽂등심이다ㅠㅜ,0.0
2,짱깨 꺼라ㅡ패쓰,0.0
3,그들의 사생활 ~ 고인이된 설리를 위해서라도 모두 조용하길 지금 누굴 탓한다고 무슨...,1.0
4,아무리 법이 뭣같아도 무슨 자격으로 개인의 신상정보를 불특정 다수에게 공개하는지 도...,1.0
...,...,...
9995,이거 보다 더한 막장의 드라마도 넋놓고 보면서 무슨?,1.0
9996,머...제네시스..올라프에디션?,1.0
9997,항상 여자로 ㅡ ㅡ 저지랄 들이야 남자들 벗겨봐라,0.0
9998,첫번째 이유가 ㄹㅇ 맞지특히나 뉴스나 기사 몇줄읽은 학생들끼리 대화해도 서로 존나 ...,0.0


In [8]:
def build_prompt(sentence: str) -> str:
    return (
        "다음 문장이 긍정인지 부정인지 판단하세요.\n\n"
        "### 문장:\n"
        f"{sentence}"
    )


In [9]:
# 텍스트 컬럼 업데이트
df["text"] = df["text"].apply(build_prompt)

In [10]:
df= df.dropna().reset_index(drop=True)

In [11]:
df['label'].value_counts()

label
1.0    4992
0.0    4983
Name: count, dtype: int64

In [12]:
valid_df=df.sample(n=500,random_state=42)

train_df=df.drop(valid_df.index)
train_df

,text,label
0,다음 문장이 긍정인지 부정인지 판단하세요.\n\n### 문장:\n이종석 한효주 나오...,0.0
1,다음 문장이 긍정인지 부정인지 판단하세요.\n\n### 문장:\n씨바알..노무노무 ...,0.0
2,다음 문장이 긍정인지 부정인지 판단하세요.\n\n### 문장:\n짱깨 꺼라ㅡ패쓰,0.0
4,다음 문장이 긍정인지 부정인지 판단하세요.\n\n### 문장:\n아무리 법이 뭣같아...,1.0
5,다음 문장이 긍정인지 부정인지 판단하세요.\n\n### 문장:\n다음도 들어와라하고...,0.0
...,...,...
9970,다음 문장이 긍정인지 부정인지 판단하세요.\n\n### 문장:\n이거 보다 더한 막...,1.0
9971,다음 문장이 긍정인지 부정인지 판단하세요.\n\n### 문장:\n머...제네시스.....,1.0
9972,다음 문장이 긍정인지 부정인지 판단하세요.\n\n### 문장:\n항상 여자로 ㅡ ㅡ...,0.0
9973,다음 문장이 긍정인지 부정인지 판단하세요.\n\n### 문장:\n첫번째 이유가 ㄹㅇ...,0.0


In [13]:
valid_df


,text,label
5825,다음 문장이 긍정인지 부정인지 판단하세요.\n\n### 문장:\n고주원 커플방송 늘...,1.0
5171,다음 문장이 긍정인지 부정인지 판단하세요.\n\n### 문장:\n야이 빙신아 천조국...,0.0
6041,다음 문장이 긍정인지 부정인지 판단하세요.\n\n### 문장:\n근대 이와중에 박나...,0.0
107,다음 문장이 긍정인지 부정인지 판단하세요.\n\n### 문장:\n우리찜니 넘조아,1.0
3422,다음 문장이 긍정인지 부정인지 판단하세요.\n\n### 문장:\n장모님과 연 끊고 ...,0.0
...,...,...
5896,다음 문장이 긍정인지 부정인지 판단하세요.\n\n### 문장:\n꺼져,0.0
6551,다음 문장이 긍정인지 부정인지 판단하세요.\n\n### 문장:\n인정할건인정해라 닥...,0.0
450,다음 문장이 긍정인지 부정인지 판단하세요.\n\n### 문장:\n탑이랑 걸렸을 때 ...,1.0
1593,다음 문장이 긍정인지 부정인지 판단하세요.\n\n### 문장:\n존예고만 어디가 안...,1.0


In [14]:
train_df=train_df.reset_index(drop=True)
valid_df=valid_df.reset_index(drop=True)


In [15]:
data= datasets.DatasetDict({
    'train': datasets.Dataset.from_pandas(train_df),
    'valid': datasets.Dataset.from_pandas(valid_df)
})

In [16]:
data

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 9475
    })
    valid: Dataset({
        features: ['text', 'label'],
        num_rows: 500
    })
})

In [17]:
import os
import numpy as np
_MAX_LEN = int(os.getenv("MAX_LEN", 512))


def get_dataset(data, tokenizer, max_len):
    def _encode(batch):
        enc = tokenizer(batch["text"],
                        truncation=True,
                        padding="max_length",
                        max_length=max_len)
        # 라벨을 int64, shape [B]
        enc["labels"] = np.array(batch["label"], dtype=np.int64)
        return enc

    tokenised = data.map(_encode, batched=True, remove_columns=["text", "label"])
    tokenised.set_format("torch",
                         columns=["input_ids", "attention_mask", "labels"])
    return tokenised

In [18]:
import torch
from transformers import Trainer, TrainingArguments, AutoTokenizer, DataCollatorWithPadding
from transformers import AutoConfig, AutoModelForSequenceClassification
from peft import LoraConfig, get_peft_model



/data/conda_envs/project1/lib/python3.11/site-packages/transformers/utils/hub.py:111: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


In [19]:
model_path='skt/kobert-base-v1'


config = AutoConfig.from_pretrained(model_path, num_labels=2, problem_type="single_label_classification")
model = AutoModelForSequenceClassification.from_pretrained(model_path, config=config)
tokenizer  = AutoTokenizer.from_pretrained(model_path, use_fast=False)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at skt/kobert-base-v1 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [20]:
model.config.id2label = {0: "none", 1: "toxic"}
model.config.label2id = {v: k for k, v in model.config.id2label.items()}

In [21]:
model

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(8002, 768, padding_idx=1)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-

In [22]:
# KoBERT 모델의 경우
targets = ["query", "key", "value"]

lora_cfg = LoraConfig(
    r=16,
    lora_alpha=16,
    lora_dropout=0.1,
    bias="none",
    task_type="SEQ_CLS",
    target_modules=targets)


model = get_peft_model(model, lora_cfg)

In [23]:
model

PeftModelForSequenceClassification(
  (base_model): LoraModel(
    (model): BertForSequenceClassification(
      (bert): BertModel(
        (embeddings): BertEmbeddings(
          (word_embeddings): Embedding(8002, 768, padding_idx=1)
          (position_embeddings): Embedding(512, 768)
          (token_type_embeddings): Embedding(2, 768)
          (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (encoder): BertEncoder(
          (layer): ModuleList(
            (0-11): 12 x BertLayer(
              (attention): BertAttention(
                (self): BertSdpaSelfAttention(
                  (query): lora.Linear(
                    (base_layer): Linear(in_features=768, out_features=768, bias=True)
                    (lora_dropout): ModuleDict(
                      (default): Dropout(p=0.1, inplace=False)
                    )
                    (lora_A): ModuleDict(
                      (default)

In [24]:
ds = get_dataset(data, tokenizer, _MAX_LEN)


Map: 100%|██████████| 500/500 [00:00<00:00, 3215.53 examples/s]


In [25]:
ds

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 9475
    })
    valid: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 500
    })
})

In [26]:
model.resize_token_embeddings(len(tokenizer))

model.config.pad_token_id = tokenizer.pad_token_id 

In [27]:
training_args = TrainingArguments(
        output_dir='model-checkpoints/kobert',
        per_device_train_batch_size=128,
        per_device_eval_batch_size=16,
        num_train_epochs=20,
        learning_rate=5e-5,
        fp16=True,
        eval_strategy="steps",
        save_strategy="steps",
        logging_steps=1,
        load_best_model_at_end=True,
        metric_for_best_model="eval_loss",
        save_steps=50,
        eval_steps=50,
    )

In [28]:
class SmartCollator(DataCollatorWithPadding):
    def __call__(self, features):
        batch = super().__call__(features)
        # BERT류만 전체 0으로 세팅
        if "token_type_ids" in batch:
            batch["token_type_ids"].zero_()  
        return batch


In [29]:
data_collator = SmartCollator(tokenizer, return_tensors="pt")

In [30]:
import evaluate
from typing import Dict, Tuple
import numpy as np

f1_metric = evaluate.load("f1")
accuracy_metric = evaluate.load("accuracy")

In [31]:
import numpy as np

def compute_metrics(eval_pred):
    logits, labels = eval_pred                      # logits: [B,2], labels: [B]

    preds  = logits.argmax(axis=-1)                 # 클래스 0/1
    labels = labels.astype(np.int64)

    acc = accuracy_metric.compute(predictions=preds, references=labels)["accuracy"]
    f1  = f1_metric.compute(predictions=preds, references=labels,
                            average="weighted")["f1"]
    return {"accuracy": acc, "f1": f1}


In [32]:
trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=ds["train"],
        eval_dataset=ds["valid"],
        data_collator=data_collator,
        compute_metrics=lambda p: compute_metrics(p),
    )


No label_names provided for model class `PeftModelForSequenceClassification`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [33]:
trainer.train()


Step,Training Loss,Validation Loss,Accuracy,F1
50,0.699500,0.691547,0.506000,0.461378
100,0.697000,0.690152,0.560000,0.543510
150,0.656200,0.684172,0.592000,0.585416
200,0.669400,0.676839,0.586000,0.553929
250,0.668700,0.640544,0.648000,0.633015
300,0.847800,0.582541,0.682000,0.675640
350,0.572100,0.561169,0.706000,0.697870
400,0.578200,0.517711,0.760000,0.755610
450,0.174900,0.462164,0.776000,0.775799
500,0.595900,0.454235,0.786000,0.785970


TrainOutput(global_step=1500, training_loss=0.5190828965951999, metrics={'train_runtime': 433.3719, 'train_samples_per_second': 437.269, 'train_steps_per_second': 3.461, 'total_flos': 5.0375484082176e+16, 'train_loss': 0.5190828965951999, 'epoch': 20.0})